# 7 Higher-order functions

This IHaskell notebook outlines a chapter from the 2nd edition of *Programming in Haskell* by Prof. Graham Hutton,
demonstrating examples and summarising some of the sections.

## 7.1 Basic concepts

In [1]:
add :: Int -> Int -> Int
add x y = x + y

In [2]:
add :: Int -> (Int -> Int)
add =  \x  -> (\y  -> x + y)

In [1]:
twice :: (a -> a) -> a -> a
twice f x = f (f x)

In [2]:
twice :: (a -> a) -> (a -> a)
twice =  \f       -> \x -> f (f x)

In [3]:
twice (*2) 3

12

In [4]:
twice reverse [1,2,3]

[1,2,3]

In [5]:
reverse [1,2,3]
reverse (reverse [1,2,3])

[3,2,1]

[1,2,3]

## 7.2 Processing lists

### map
Defining `map` using list comprehension:
```haskell
map :: (a -> b) -> [a] -> [b]
map f xs = [f x | x <- xs]
```
Defining `map` as a recursive function:
```haskell
map :: (a -> b) -> [a] -> [b]
map f []     = []
map f (x:xs) = f x : map f xs
```

In [6]:
map (+1) [1,3,5,7]

[2,4,6,8]

In [7]:
map even [1,2,3,4]

[False,True,False,True]

In [8]:
map reverse ["abc","def","ghi"]

["cba","fed","ihg"]

In [11]:
:type [ [1,2,3], [4,5] ]
:type (map (+1))

[ [1,2,3], [4,5] ] :: forall a. Num a => [[a]]

(map (+1)) :: forall b. Num b => [b] -> [b]

In [13]:
map (+1) [1,2,3]
map (+1) [4,5]

[2,3,4]

[5,6]

In [15]:
map (map (+1)) [ [1,2,3], [4,5] ]

[[2,3,4],[5,6]]

In [16]:
[ [x+1 | x <- xs]   | xs <- [[1,2,3],[4,5]] ]

[[2,3,4],[5,6]]

In [17]:
map (map (+1)) [[1,2,3],[4,5]]
  == {- applying the outer map -}          [[2,3,4],[5,6]]
[map (+1) [1,2,3], map (+1) [4,5]]
  == {- applying the inner map -}
  [[2,3,4],[5,6]]

True

True

### filter
Defining `filter` using list comprehension:
```haskell
filter :: (a -> Bool) -> [a] -> [a]
filter p xs = [x | x <- xs, p x]
```
Defining `filter` as a recursive function:
```haskell
filter p []     = []
filter p (x:xs)
    | p x       = x : filter p xs
    | otherwise = filter p xs
```

In [9]:
filter even [1,2,3,4,5]

[2,4]

### other higher-order functions for processing lists

In [10]:
all even [0,2,4,6,8] -- 모든 원소가 조건을 만족하는지
all even [2,4,6,7,8]

True

False

In [11]:
any odd [0,2,4,6,8] -- 조건을 만족하는 원소가 한개라도 존재하는지
any odd [2,4,6,7,8]

False

True

In [12]:
take 3 [1,2,3,4,5]
drop 3 [1,2,3,4,5]

[1,2,3]

[4,5]

In [23]:
takeWhile even [2,4,6,7,8,9]
takeWhile even [1,4,6,7,8,9]

[2,4,6]

[]

In [25]:
dropWhile odd [1,3,5,6,7]
dropWhile odd [2,3,5,6,7]

[6,7]

[2,3,5,6,7]

In [31]:
-- 문자열에서 첫번째 단어만을 뽑은 문자열을 계산하는 함수
import Data.Char (isSpace)

dropWhile isSpace "  hello  world  nice"
takeWhile (not . isSpace) (dropWhile isSpace "  hello  world  nice")

firstWord = takeWhile (not . isSpace) . dropWhile isSpace

firstWord "  hello  world  nice"

"hello  world  nice"

"hello"

"hello"

## 7.3 The `foldr` function

A common simple pattern of recursion over lists:
```haskell
f []      = v
f (x:xs)  = x ⚝ f xs
```
The `foldr` function encapsulates this pattern of recursion:
```haskell
f = foldr (⚝) v
```
The behavior of `foldr` can be summarized as follows:
```haskell
foldr (⚝) v [x0,x1,....,xn]
  == x0 ⚝ (x1 ⚝ ( ....  (xn ⚝ v)....)) 
```
The name *fold right* reflects the use of an operator that is assumed to associate to the right.

----
Let us revisit the common simple pattern of recursion over lists:
```haskell
f []      = v
f (x:xs)  = x ⚝ f xs
```

In [32]:
foldr :: (a -> b -> b) -> b -> [a] -> b
foldr (⚝) v []     = v
foldr (⚝) v (x:xs) = x ⚝ (foldr (⚝) v xs)

In [13]:
product [2,3,4]

24

In [34]:
-- v = 0        (⚝) = (+)
sum []     = 0
sum (x:xs) = x + sum xs
-- v = 1        (⚝) = (*)
product []     = 1
product (x:xs) = x * product xs
-- v = False    (⚝) = (||)
or []     = False
or (x:xs) = x || or xs
-- v = True     (⚝) = (&&)
and []     = True
and (x:xs) = x && and xs

In [35]:
sum     = foldr (+)  0
product = foldr (*)  1
or      = foldr (||) False
and     = foldr (&&) True

In [39]:
-- 똑같은 리스트를 떼어냈다가 새로 만들어내는 계산
copylist = foldr (:) []

In [42]:
copylist [1,2,3,4]

[1,2,3,4]

In [41]:
[1,2,3,4] == 1 : (2 : (3 : (4 : [])))

Line 1: Use list literal
Found:
1 : (2 : (3 : (4 : [])))
Why not:
[1, 2, 3, 4]

True

In [43]:
concat [ [1,2], [3,4], [5], [6,7,8] ]

[1,2,3,4,5,6,7,8]

In [45]:
concat xss = [x | xs<-xss, x<-xs]

In [46]:
concat [ [1,2], [3,4] ]

[1,2,3,4]

In [48]:
concat :: [[a]] -> [a]
concat []       = []
concat (xs:xss) = xs ++ concat xss

In [49]:
concat [ [1,2], [3,4] ]

[1,2,3,4]

In [50]:
concat = foldr (++) []

In [51]:
concat [ [1,2], [3,4] ]

[1,2,3,4]

## 7.4 The `foldl` function

Another common simple pattern of recursion over lists:
```haskell
f v []      = v
f v (x:xs)  = f (v ⚝ x) xs
```
The `foldr` function encapsulates this pattern of recursion:
```haskell
f = foldl (⚝) v
```
The behavior of `foldl` can be summarized as follows:
```haskell
foldl (⚝) v [x0,x1,....,xn]
  == (....((v ⚝ x0) ⚝ x1) .... ) ⚝ xn 
```
The name *fold left* reflects the use of an operator that is assumed to associate to the left.

In [54]:
foldl :: (b -> a -> b) -> b -> [a] -> b
foldl (⚝) v []     = v
foldl (⚝) v (x:xs) = foldl (⚝) (v ⚝ x) xs

In [56]:
-- ((((100 - 1) - 2) - 3) - 4) - 5 
foldl (-) 100 [1,2,3,4,5]

85

## 7.5 The composition operator

* $~f \,\circ\, g~$ in mathematics
* `f . g` in Haskell

```haskell
(.) :: (b -> c) -> (a -> b) -> (a -> c)
f . g = \x -> f (g x)
```

The identity function `id = \x -> x` is
the identity for the composition operator `(.)`.
That is,
```haskell
   id . f  ==  f  ==  f . id
```

In [58]:
compose :: [(a -> a)] -> (a -> a)
compose = foldr (.) id

In [59]:
compose [(*4),(+1),(^2)] 2

20


## 7.6 Binary string transmitter

## 7.7 Voting algorithms

----

In [ ]:
:opt no-pager -- to inline :info